In [3]:
import argparse
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import json

def load_data(data):
    iris = pd.read_csv(data, sep=',')
    print(iris.shape)
    return iris

def get_train_test_data(iris):

    encode = LabelEncoder()
    iris.Species = encode.fit_transform(iris.Species)

    train , test = train_test_split(iris, test_size=0.2, random_state=0)
    print('shape of training data : ', train.shape)
    print('shape of testing data', test.shape)


    X_train = train.drop(columns=['Species'], axis=1)
    y_train = train['Species']
    X_test = test.drop(columns=['Species'], axis=1)
    y_test = test['Species']

    return X_train, X_test, y_train, y_test

def evaluation(y_test,predict):
    print("accuracy : ",accuracy_score(y_test,predict))
    
    accuracy = accuracy_score(y_test,predict)
    
    metrics = {
        'metrics': [{
            'name': 'accuracy-score',
            'nuberValue': accuracy,
            'format': "%",
        }]
    }
    
    with open('./accuracy.json','w') as f:
        json.dump(accuracy,f)
    
    with open('./mlpipeline-metrics.json','w') as f:
        json.dump(metrics,f)


if __name__ == "__main__":
    
    argument_parser = argparse.ArgumentParser()

    argument_parser.add_argument(
        '--data',
        type=str,
        default='./Iris.csv',
        help="Input data csv"
    )

    args = argument_parser.parse_args()
    iris = args.data
    iris = load_data(iris)

    X_train, X_test, y_train, y_test = get_train_test_data(iris)

    model = LogisticRegression(max_iter=5000)
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    print('\nAccuracy Score on test data : ')
    print(accuracy_score(y_test, predict))
    evaluation(y_test,predict)
    

usage: ipykernel_launcher.py [-h] [--data DATA]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/jovyan/.local/share/jupyter/runtime/kernel-d64992cd-7fea-4625-8314-541624ff8be7.json


SystemExit: 2

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import kfp
import kfp.components as comp
from kfp import dsl

def print_op(msg):
    
    return dsl.ContainerOp(
        name='Print',
        command=['ehco',msg],
    )



@dsl.pipeline(
    name='iris-train',
    description='training iris-dataset demo'
)

def pipeline():
    add_p = dsl.ContainerOp(
        name="load iris data pipeline",
        arguments=[
            '--data_path', './Iris.csv'
        ],
        file_outputs={'iris' : '/iris.csv'}
    )

    ml = dsl.ContainerOp(
        name="training pipeline",
        arguments=[
            '--data', add_p.outputs['iris']
        ],
        file_outputs={
            'accuracy' : './accuracy.json',
            'mlpipeline-metrics': './mlpipeline-metrics.json'
        }
    )

    ml.after(add_p)
    baseline = 0.7
    
    with dsl.Condition(ml.outputs['accuracy'] > baseline) as check_condition:
        print_op(f"accuracy는 {ml.outputs['accuracy']}로 accuracy baseline인 {baseline}보다 크다.")
    
    with dsl.Condition(ml.outputs['accuracy'] < baseline) as check_condition:
        print_op(f"accuracy는 {ml.outputs['accuracy']}로 accuracy baseline인 {baseline}보다 작다.")

    

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline, 'iris-pipeline' + ".tar.gz")

TypeError: __init__() missing 1 required positional argument: 'image'